In [1]:
import pandas as pd
import matplotlib as mp
import numpy as np
from datetime import timedelta
import datetime as dt
import gc

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb

%matplotlib inline

#Root de archivos de datos
root_dir = '../data/'

#Lee menos data si es testing
is_testing = True

# Cantidad minima de aparicion de equipos en un dataframe
min_devices = 10

# Identificador unico de equipos
device_uuid = ['ref_type','ref_hash']

# Ventanas de tiempo
windows = pd.DataFrame({
'window_nr':[1,2,3,4],
'n': 1,
'min_date': dt.datetime(2019,4,18),
'max_date': [dt.datetime(2019,4,21),dt.datetime(2019,4,22),dt.datetime(2019,4,23),dt.datetime(2019,4,24)]
})

Prediccion de Tiempos de Arribo
===============================

Se utilizan los distintos dataframes para armar un set de features que sirvan para predecir el tiempo hasta la aparicion de un dispositivo nuevamente. 

Este notebook tiene la siguiente estructura: 

1. Lectura de los Dataframes
2. Arreglo de los datos
3. Armado de Features
4. Armado de labels
5. Armado del set de pruebas
6. Training del Modelo Predictivo
7. Predicciones
8. Evaluacion 

------------------------------
## 1. Lectura de los Dataframes

Se realiza la carga de los dataframes en memoria para el armado del modelo predictivo. La lectura se hace optimizando los tipos de datos a fin de utilizar la menor cantidad de memoria posible.

In [2]:
#Optimizado para menos memoria
auction_dtypes = {
    'ref_type_id': np.int8,
    'device_id': np.int64,
    'source_id': np.int8
}

auctions = pd.read_csv(root_dir + 'auctions.csv.gzip',
                       compression = 'gzip',
                       dtype = auction_dtypes,
                       parse_dates = ['date'])
auctions.rename({'device_id':'ref_hash',
                 'ref_type_id':'ref_type'}, inplace=True, axis='columns')
auctions['n'] = 1

# Para hacer pruebas ocupando menos memoria se hace un sampleo aleatorio de la mitad del dataframe y se elimina el resto
if is_testing:
    auctions = auctions.sample(frac=.30)
    print("Is Testing, #records:", auctions['n'].sum())
else:
    print("Not Testing, #records:", auctions['n'].sum())


Is Testing, #records: 14222858


In [60]:
installs_dtypes = {
    'application_id': np.int32,
    'ref_type': np.int64,
    'ref_hash': np.int64, 
    #'click_hash':'category',
    'attributed': 'category',
    'implicit': 'category',
    #'device_countrycode': 'object', 
    'device_brand': 'object',
    'device_model': 'object', 
    'session_user_agent': 'object', 
    'user_agent': 'object', 
    'event_uuid':'object',
    'kind': 'object',
    'wifi': 'object', 
    'trans_id': 'object', 
    #'ip_address':'object', 
    'device_language': 'object'
}
install_cols = list(installs_dtypes.keys()) + ['created']
installs = pd.read_csv(root_dir + 'installs.csv.gzip', 
                       compression='gzip', 
                       usecols=install_cols,
                       dtype= installs_dtypes,
                      parse_dates=['created'])

installs['n'] = 1
print('#records:', installs['n'].sum())

#records: 481511


In [63]:
installs.loc[installs['ref_type'] == 1891515180541284343, 'ref_type'] = 1
installs.loc[installs['ref_type'] == 1494519392962156891, 'ref_type'] = 7

In [4]:
clicks_dtypes = {
    'advertiser_id': np.int8, 
    'action_id': np.int32, 
    'source_id': np.int8, 
    'latitude' : np.float64, 
    'longitude': np.float64, 
    'wifi_connection': np.bool, 
    'carrier_id': np.int16, 
    'trans_id': 'object',
    'os_minor':'object', 
    #'agent_device' : 'category', 
    #'os_major': 'category', 
    'specs_brand': 'category', 
    #'brand': np.int8,
    'timeToClick': np.float64, 
    #'touchX': np.float64, 
    #'touchY': np.float64, 
    'ref_type':np.int64, 
    'ref_hash':np.int64,
    'created' : 'object'
}
clicks = pd.read_csv(root_dir + 'clicks.csv.gzip', 
                     compression='gzip',
                     low_memory=False,
                     parse_dates=['created'])
clicks['n'] = 1
print('#records:', clicks['n'].sum())

#records: 64296


In [5]:
events_dtypes = {
    'event_id': np.int64,
    'ref_type': np.int64,
    'ref_hash': np.int64,
    'application_id': np.int64,
    'attributed': np.bool,
    'device_os_version': 'object',
    'device_brand': 'object',
    'device_model': 'object',
    'device_city': 'object',
    'session_user_agent': 'object',
    'trans_id': 'category',
    'user_agent': 'object',
    'event_uuid': 'object',
    'carrier': 'object',
    'kind': 'object',
    'device_os': 'object',
    'wifi': np.bool,
    'connection_type': 'object',
    #'ip_address': np.int64,
    #'device_language': 'category'
}
events_cols = list(events_dtypes.keys()) + ['date']
events = pd.read_csv(root_dir + 'events.csv.gzip', 
                     compression='gzip',
                     dtype=events_dtypes,
                     usecols=events_cols,
                     parse_dates=['date'])
events['n'] = 1
# Para hacer pruebas ocupando menos memoria se hace un sampleo aleatorio de la mitad del dataframe y se elimina el resto
if is_testing:
    events = events.sample(frac=.30)
    print("Is Testing, #records:", events['n'].sum())
else:
    print("Not Testing, #records:", events['n'].sum())

Is Testing, #records: 2323374


In [6]:
# Labels a submitir con las predicciones

to_predict = pd.read_csv(root_dir + 'target_final_competencia_revamped.csv')
to_predict = to_predict.apply(lambda x: np.int64(x['ref_hash'][0:x['ref_hash'].find('_')]), axis='columns').drop_duplicates().to_frame()
to_predict.columns = ['ref_hash']

---
## 2. Arreglo de los Datos

### 2.1 Arreglo de datos de Auctions


In [7]:
#Elimino los registros con menos de un minimo de entradas, ya que no hay mucho que predecir en estos casos
orig_count = auctions['n'].sum()
auctions = auctions.groupby(device_uuid, sort = False).filter(lambda x: x['n'].sum() >= min_devices) 
last_count = auctions['n'].sum()
print('Eliminados:', orig_count-last_count)

Eliminados: 791731


In [8]:
auctions.sort_values(by=(device_uuid+['date']), inplace=True)
auctions['next_date'] = auctions.groupby(device_uuid, as_index = False, sort=False)['date'].transform(lambda x: x.shift(-1))
auctions = auctions.loc[(~auctions['next_date'].isnull())]
auctions['secs_to_next'] = (auctions['next_date'] - auctions['date']).transform(lambda x: round(x.total_seconds()))

In [9]:
auctions['n'].sum()

13254235

In [10]:
auctions['date'].min()

Timestamp('2019-04-18 00:00:00.154022')

In [11]:
auctions['date'].max()

Timestamp('2019-04-26 23:59:57.806699')

In [12]:
auctions.head()

,date,ref_hash,ref_type,source_id,n,next_date,secs_to_next
30900602,2019-04-19 19:40:28.465866,41863526108385,1,8,1,2019-04-20 02:52:26.892880,25918
14875535,2019-04-20 02:52:26.892880,41863526108385,1,3,1,2019-04-20 03:06:01.675788,815
45613542,2019-04-20 03:06:01.675788,41863526108385,1,3,1,2019-04-20 03:08:57.388160,176
42215232,2019-04-20 03:08:57.388160,41863526108385,1,3,1,2019-04-20 03:11:41.011671,164
12690007,2019-04-20 03:11:41.011671,41863526108385,1,3,1,2019-04-20 03:20:23.061467,522


### 2.2 Arreglo de datos de Installs

In [13]:
installs['kind'] = installs['kind'].str.replace(' ','_')
installs['kind'] = installs['kind'].str.replace('af_app_open ','af_app_opened')
installs['kind'] = installs['kind'].str.replace('af_app_opend','af_app_opened')
installs['kind'] = installs['kind'].str.lower()

In [14]:
installs.isnull().sum()

created                    0
application_id             0
ref_type                   0
ref_hash                   0
attributed                 0
implicit                   0
device_brand          205068
device_model           26892
session_user_agent     14839
user_agent            150743
event_uuid            378343
kind                  378343
wifi                  186682
trans_id              472578
device_language        27577
n                          0
dtype: int64

In [15]:
for i in ['device_brand','device_model','session_user_agent','user_agent','kind','wifi','device_language']:
    installs[i] = installs[i].fillna('unknown')
installs['device_brand'] = installs['device_brand'].astype('category')
installs['device_model'] = installs['device_model'].astype('category')
installs['session_user_agent'] = installs['session_user_agent'].astype('category')
installs['user_agent'] = installs['user_agent'].astype('category')
installs['kind'] = installs['kind'].astype('category')
installs['wifi'] = installs['wifi'].astype('category')
installs['device_language'] = installs['device_language'].astype('category')

In [16]:
installs.head().transpose()

,0,1,2,3,4
created,2019-04-24 06:23:29.495000,2019-04-24 02:06:01.032000,2019-04-20 10:15:36.274000,2019-04-20 21:56:47.151000,2019-04-20 22:40:41.239000
application_id,1,1,1,1,1
ref_type,1494519392962156891,1494519392962156891,1494519392962156891,1494519392962156891,1494519392962156891
ref_hash,4716708407362582887,7143568733100935872,5230323462636548010,5097163995161606833,6328027616411983332
attributed,False,False,False,False,False
implicit,True,False,True,True,False
device_brand,unknown,unknown,unknown,unknown,unknown
device_model,3.739127126472163e+17,7.80553892759877e+18,8.355495513718673e+18,2.3557720913769155e+18,6.156971151807135e+18
session_user_agent,adjust.com,adjust.com,adjust.com,adjust.com,adjust.com
user_agent,unknown,unknown,unknown,unknown,unknown


### 2.3 Arreglo de datos de Clicks

In [17]:
clicks.touchX = clicks.touchX.apply(lambda x: np.float64(x))
clicks.touchY = clicks.touchY.apply(lambda x: np.float64(x))
clicks['created'] = clicks['created'].dt.tz_localize(None)

In [18]:
for i in ['carrier_id','os_minor', 'agent_device', 'brand', 'os_major']:
    clicks[i] = clicks[i].fillna('unknown').astype('category')

In [19]:
clicks.isnull().sum()

advertiser_id          0
action_id          64289
source_id              0
created                0
country_code           0
latitude               0
longitude              0
wifi_connection        0
carrier_id             0
trans_id               0
os_minor               0
agent_device           0
os_major               0
specs_brand            0
brand                  0
timeToClick        26118
touchX             20618
touchY             20618
ref_type               0
ref_hash               0
n                      0
dtype: int64

### 2.4 Arreglo de datos de Events

In [20]:
for i in ['device_os_version','device_brand','device_model','device_city','session_user_agent','user_agent','carrier','kind','device_os','connection_type']:
    events[i] = events[i].fillna('unknown').astype('category')

In [21]:
events.isnull().sum()

date                        0
event_id                    0
ref_type                    0
ref_hash                    0
application_id              0
attributed                  0
device_os_version           0
device_brand                0
device_model                0
device_city                 0
session_user_agent          0
trans_id              2312088
user_agent                  0
event_uuid               8851
carrier                     0
kind                        0
device_os                   0
wifi                        0
connection_type             0
n                           0
dtype: int64

---
Separado de las ventanas de tiempo
----------------------------------

### Separo las semanas de entrenamiento

Se pueden utilizar distintos metodos: 

- Se arman ventanas de 3 días para predecir 3 dias
- Se arman ventanas de n-1 dias para predecir n a n+2 dias.

Para el caso de estudio se utiliza la opcion de maximizar la cantidad de datos para predecir cada ventana.

Los datos pueden entonces estar en las siguientes ventanas y deben solo usarse para predecir la ventana correspondiente en los sets de training.

Las ventanas de los sets de entrenamiento son: 

1. 21 al 23
2. 22 al 24
3. 23 al 25
4. 24 al 26

Entonces las ventanas armadas serán:
1. 18 al 20
2. 18 al 21
3. 18 al 22
4. 18 al 23

---
## 3. Armado de Features

A continuacion se comienzan a extraer los distintos features que formaran el set de entrenamiento. 
Se cruza cada ventana con los labels que se desean predecir. 

Para entrenar el set debe decidirse como utilizar los datos de las distintas ventanas. 

- Una opcion será mezclar todos los datos pero hay que decidir que hacer con los equipos que aparecen mas de una vez. 

- La segunda opcion es entrenar 4 modelos distintos y verificar que haya una mejora en todos ellos.

In [22]:
# Entreno con ventana de 18 al 23 y valido con ventana del 24 al 26
cutoff_date = windows.set_index('window_nr').loc[4,'max_date']

train_auctions = auctions.loc[auctions['date'] < cutoff_date]
train_installs = installs.loc[installs['created'] < cutoff_date]
train_events = events.loc[events['date'] < cutoff_date]
train_clicks = clicks.loc[clicks['created'] < cutoff_date]

In [45]:
labels_installs = installs.loc[installs['created'] >= cutoff_date]

In [23]:
devices = train_auctions[device_uuid].drop_duplicates()
devices['ref_hash'].count()

166807

### Armado de tiempo entre arribos

Se desea saber el tiempo promedio entre arribos de los dispositivos a las encuestas.

In [24]:
#filtered = auctions.loc[auctions['secs_to_next'] < 120]

In [30]:
temp = train_auctions.groupby(device_uuid, as_index='False')['secs_to_next'].mean().to_frame()
temp.columns = ['secs_to_next_mean']

devices = devices.merge(temp, how='left', on=device_uuid).set_index(device_uuid)

In [31]:
devices.head()

secs_to_next_mean
ref_type ref_hash                          
1        41863526108385         2422.538462
         186034136943920       18332.304348
         345999128501141       32896.714286
         360710529886978       84419.166667
         416301579449694       35260.600000

### Tiempo desde ultima aparicion

In [32]:
max_date = train_auctions['date'].max()

In [33]:
time = train_auctions.groupby(device_uuid).apply(lambda x: round((max_date - x['date'].max()).total_seconds())).to_frame()
time.columns = ['secs_since_last_arrival']

In [35]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [36]:
devices.head()

secs_to_next_mean  secs_since_last_arrival_x  \
ref_type ref_hash                                                        
1        41863526108385         2422.538462                     329859   
         186034136943920       18332.304348                      37583   
         345999128501141       32896.714286                       6225   
         360710529886978       84419.166667                     294147   
         416301579449694       35260.600000                     165353   

                          secs_since_last_arrival_y  
ref_type ref_hash                                    
1        41863526108385                      329859  
         186034136943920                      37583  
         345999128501141                       6225  
         360710529886978                     294147  
         416301579449694                     165353

### Cantidad de apariciones en encuestas 

In [38]:
amount_auctions = train_auctions.groupby(device_uuid)['n'].count().to_frame()
amount_auctions.columns = ['auctions_total']
devices = devices.merge(amount_auctions,how = 'left', left_index=True, right_index=True)
devices.head()

secs_to_next_mean  secs_since_last_arrival_x  \
ref_type ref_hash                                                        
1        41863526108385         2422.538462                     329859   
         186034136943920       18332.304348                      37583   
         345999128501141       32896.714286                       6225   
         360710529886978       84419.166667                     294147   
         416301579449694       35260.600000                     165353   

                          secs_since_last_arrival_y  auctions_total  
ref_type ref_hash                                                    
1        41863526108385                      329859              13  
         186034136943920                      37583              23  
         345999128501141                       6225              14  
         360710529886978                     294147               6  
         416301579449694                     165353              10

In [39]:
amount_last_auctions = train_auctions.groupby(device_uuid).apply(lambda x: x.loc[x['date'] > (max_date - timedelta(hours=1)),'n'].count()).to_frame()
amount_last_auctions.columns = ['auctions_last_hour']
devices = devices.merge(amount_last_auctions, how='left', left_index=True, right_index=True)
devices.head()

secs_to_next_mean  secs_since_last_arrival_x  \
ref_type ref_hash                                                        
1        41863526108385         2422.538462                     329859   
         186034136943920       18332.304348                      37583   
         345999128501141       32896.714286                       6225   
         360710529886978       84419.166667                     294147   
         416301579449694       35260.600000                     165353   

                          secs_since_last_arrival_y  auctions_total  \
ref_type ref_hash                                                     
1        41863526108385                      329859              13   
         186034136943920                      37583              23   
         345999128501141                       6225              14   
         360710529886978                     294147               6   
         416301579449694                     165353              10   

                          auctions_last_hour  
ref_type ref_hash                             
1        41863526108385                    0  
         186034136943920                   0  
         345999128501141                   0  
         360710529886978                   0  
         416301579449694                   0

### Secuencia de ultimos 5 eventos del dispositivo

---
## 4. Armado de Labels 

### 4.1. Prediccion de Installs

In [65]:
all_ref_hash = devices.reset_index()[device_uuid].drop_duplicates()
len(all_ref_hash)

166807

In [67]:
installs['ref_type'].value_counts()

1    398906
7     82605
Name: ref_type, dtype: int64

In [76]:
window = 3
j=1
installs_training = pd.DataFrame()

for i in range(21,25): #[19, 20, 21, 22, 23, 24]
    print("Ventana: ", j)
    temp = pd.DataFrame()
    
    temp[device_uuid + ['date_install']] = installs.loc[(installs['created'].dt.day >= i) & (installs['created'].dt.day < (i+window))]\
                                                    .groupby(device_uuid, as_index=False)['created'].min()
    temp['window_nr'] = j
    temp['window_date_start'] = dt.datetime(2019,4,i)
    temp['secs_to_install'] = (temp['date_install']-temp['window_date_start']).transform(lambda x: x.total_seconds())
    print("  Encontrados: ", len(temp))
    installs_training = installs_training.append(temp,sort=True)
    
    #Agrego los que no se encontraron en esta ventana pero si estan en installs (aparecen en alguna)
    temp = all_ref_hash.merge(installs_training.loc[installs_training['window_nr'] == j], how='left',on=device_uuid)
    temp = temp.loc[temp['window_nr'].isnull()]
    temp['window_nr'] = j
    temp['window_date_start'] = dt.datetime(2019,4,i)
    temp['date_install'] = dt.datetime(2019,4,i+window)
    temp['secs_to_install'] = (temp['date_install']-temp['window_date_start']).transform(lambda x: x.total_seconds())
    print("  Agregados sin installs en esta ventana: ", len(temp))
    installs_training = installs_training.append(temp,sort=True)
    
    #Agrego los que no se encontraron en install pero se deben predecir conversiones
    #temp = to_predict.merge(installs_training.loc[installs_training['window_nr'] == j], how='left', on='ref_hash')
    #temp = temp.loc[temp['window_nr'].isnull()]
    #print("  Agregados sin installs pero en lista a predecir: ", len(temp))
    #temp['window_nr'] = j
    #temp['window_date_start'] = dt.datetime(2019,4,i)
    #temp['date_install'] = dt.datetime(2019,4,i+window)
    #temp['secs_to_install'] = (temp['date_install']-temp['window_date_start']).transform(lambda x: x.total_seconds())
    #installs_training = installs_training.append(temp,sort=True)
    
    j = j+1

sorted_installs = installs_training.sort_values(device_uuid + ['window_nr'])

Ventana:  1
  Encontrados:  133339
  Agregados sin installs en esta ventana:  136817
Ventana:  2
  Encontrados:  132890
  Agregados sin installs en esta ventana:  138163
Ventana:  3
  Encontrados:  133920
  Agregados sin installs en esta ventana:  139592
Ventana:  4
  Encontrados:  138336
  Agregados sin installs en esta ventana:  140101


In [77]:
sorted_installs['window_nr'].value_counts()

4    278437
3    273512
2    271053
1    270156
Name: window_nr, dtype: int64

---
## 5. Training 

In [83]:
training_set = devices.merge(sorted_installs.loc[sorted_installs['window_nr'] == 4, device_uuid+['secs_to_install']].set_index(device_uuid), left_index=True,right_index=True, how='left')

In [84]:
training_set.isnull().sum()

secs_to_next_mean            0
secs_since_last_arrival_x    0
secs_since_last_arrival_y    0
auctions_total               0
auctions_last_hour           0
secs_to_install              0
dtype: int64

In [87]:
x=devices
y=training_set['secs_to_install']

x_train, x_test, y_train, y_test =  train_test_split(x,y)

In [88]:
regr = RandomForestRegressor(random_state=0, n_estimators=100)
regr.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

---
## 6. Prediccion

In [89]:
y_predicted = regr.predict(x_test)

---
## 7. Validacion

In [90]:
mean_squared_error(y_test,y_predicted)

3400688504.3380723

In [91]:
regr.feature_importances_

array([0.41384513, 0.22300098, 0.22222123, 0.12797089, 0.01296177])

In [92]:
x_test.columns

Index(['secs_to_next_mean', 'secs_since_last_arrival_x',
       'secs_since_last_arrival_y', 'auctions_total', 'auctions_last_hour'],
      dtype='object')